In [3]:
library(dplyr)
library(Seurat)
library(ggplot2)
library(sctransform)
library(scater)
library(reticulate)
library('Biobase')
library(pheatmap)
library(gplots)
library('hdf5r')

In [4]:
packageVersion('Seurat')

[1] ‘3.0.1’

# Integration of SC-islet snATAC-seq and scRNA-seq data

## scRNA-seq data

In [24]:
test.data <- Matrix::readMM('/oasis/tscc/scratch/gaw006/share/SCislet/SC_RNA.mtx')
test.gene=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/SC_RNA.genes')
test.cell=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/SC_RNA.barcodes')
test.data=Matrix::t(test.data)

rownames(test.data)=test.gene$V1
colnames(test.data)=test.cell$V1
dim(test.data)

[1] 32738 25686

In [33]:
#B=read.csv('/projects/sanderlab/gaowei/Han/merge_final/matrix/RNAprimary_anno.csv')
B=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/SC_RNA_barcode.csv')

barcode_all=as.character(B$index)
barcode_temp=barcode_all
B_temp=B

test.data1=test.data[,colnames(test.data) %in% barcode_temp]
dim(test.data1)
all(colnames(test.data1)==as.character(B_temp$X))

primary_RNA <- CreateSeuratObject(counts = test.data1, assay = "RNA", project = "10x_RNA")
primary_RNA

primary_RNA[["leiden"]] <- as.character(B_temp$leiden)
primary_RNA[["donor"]] <- as.character(B_temp$donor)
#primary_RNA[["anno"]] <- as.character(B_temp$anno)
#primary_RNA[["anno1"]] <- as.character(B_temp$anno1)
#primary_RNA[["anno2"]] <- as.character(B_temp$anno2)

[1] 32738 25686

[1] TRUE

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”

An object of class Seurat 
32738 features across 25686 samples within 1 assay 
Active assay: RNA (32738 features)

In [35]:
primary_RNA[["percent.mt"]] <- PercentageFeatureSet(primary_RNA, pattern = "^MT-")

primary_RNA <- NormalizeData(primary_RNA, normalization.method = "LogNormalize", scale.factor = 10000)
primary_RNA <- FindVariableFeatures(primary_RNA, selection.method = "vst", nfeatures = 2000)


In [36]:
primary_RNA@meta.data$log_count=log(primary_RNA@meta.data$nCount_RNA)
primary_RNA <- ScaleData(primary_RNA, vars.to.regress = c("percent.mt","log_count"))


Regressing out percent.mt, log_count
Centering and scaling data matrix


In [37]:
primary_RNA <- RunPCA(primary_RNA, features = VariableFeatures(object = primary_RNA))

PC_ 1 
Positive:  FN1, LDHB, GSTP1, CLDN6, FBLN1, SERPINF1, PHGDH, IGFBP2, ZFP36L1, KRT18 
	   COL2A1, LYPD6B, SOX9, SPINK1, NASP, MGST1, CNN3, APOE, CSRP2, MDK 
	   REST, ANGPT2, FRZB, TUBB, KRT8, HMGB1, HMGA1, TSPAN6, MFAP2, S100A10 
Negative:  CHGA, CRYBA2, NEUROD1, SCG3, MAFB, INS, PTPRN, STMN2, KCNJ6, SCG5 
	   SLC7A8, KIAA1244, KIF5C, KCNK16, INSM1, TMED8, ATP2A3, SYT7, CACNA1A, NKX2-2 
	   MAP1B, FEV, CBX6, CELF3, CELF4, MT-ND2, ACVR1C, SCGN, CPEB4, ERO1LB 
PC_ 2 
Positive:  TTR, CLU, VEGFA, GCG, ASPH, ABCC8, INS, IFI6, SLC7A2, RPS6KA3 
	   NCAM1, DPP4, CBX6, KIAA1244, ISL1, LMO2, SLC7A8, SLC30A8, ETV1, FCGRT 
	   KIAA1324, SPTSSB, RAB7L1, SLC22A17, CTSZ, UTRN, PAX6, PALLD, ZFP36L2, JUN 
Negative:  RBP1, NEUROG3, KRT19, APOA1, THSD4, PRDX1, HIST1H2BK, LRRTM1, TAGLN3, C9orf16 
	   RP11-123K19.1, IHH, AC068535.3, PAX4, AMIGO2, FAM181B, SCG2, C22orf42, NNAT, FABP5 
	   C11orf87, HPCA, CDKN1C, MDK, HEPACAM2, RGS2, HES6, BAIAP2L2, TUBA1A, GUCA2B 
PC_ 3 
Positive:  B3GAT1, NKX6-1, MNX

In [38]:
primary_RNA <- FindNeighbors(primary_RNA, dims = 1:20)
primary_RNA <- FindClusters(primary_RNA, resolution = 0.5)
primary_RNA <- RunUMAP(primary_RNA, dims = 1:20)

Computing nearest neighbor graph
Computing SNN


Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 25686
Number of edges: 875427

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9291
Number of communities: 15
Elapsed time: 5 seconds


In [39]:
U=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/SC_scRNA_UMAP.csv')
U[1:3,]

index,UMAP_1,UMAP_2
D11_AAACCCAAGAACTGAT-1,9.486988,7.8144700
D11_AAACCCAAGACATCAA-1,6.949167,-0.5240627
D11_AAACCCACAAGAGATT-1,7.134753,-0.5965236


In [40]:
U=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/SC_scRNA_UMAP.csv')
U_temp=U[as.character(U$index) %in% as.character(B_temp$index),]
all(as.character(U_temp$index)==as.character(B_temp$index))

primary_RNA[["UMAP1"]] <- as.numeric(U_temp$UMAP_1)
primary_RNA[["UMAP2"]] <- as.numeric(U_temp$UMAP_2)

[1] TRUE

In [41]:
primary_RNA@reductions$umap@cell.embeddings[,1] <- primary_RNA$UMAP1
primary_RNA@reductions$umap@cell.embeddings[,2] <- primary_RNA$UMAP2

In [42]:
#saveRDS(primary_RNA, file = "/oasis/tscc/scratch/gaw006/share/Han/Seurat_obj/new/SC_RNA.rds")

## snATAC-seq data

In [43]:
test.data <- Matrix::readMM('/oasis/tscc/scratch/gaw006/share/SCislet/matrix_500/all_500_b.mtx')
test.data@x[test.data@x > 0] <- 1
test.gene=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/all_500_b.genes')
test.cell=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/all_500_b.barcodes')
test.data=Matrix::t(test.data)

rownames(test.data)=test.gene$V1
colnames(test.data)=test.cell$V1
dim(test.data)


[1] 462765  65255

In [44]:
B=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/SC_ATAC_barcode.csv')
barcode_all=as.character(B$index)
B[1:3,]

barcode_temp=colnames(test.data)
B_temp=B[barcode_all %in% barcode_temp,]

test.data=test.data[,colnames(test.data) %in% barcode_temp]
all(colnames(test.data)==as.character(B_temp$index))


index,batch,donor,duplicated_reads,frac_duplicated_reads,frac_mito_reads,frac_promoters_used,frac_reads_in_peaks,frac_reads_in_promoters,log10_n_counts,⋯,n_peaks,norm_log_counts,reads_in_peaks,reads_in_promoters,total_sequenced_reads,tss_used,unique_mito_reads,unique_usable_reads,leiden,anno
MM129_AAACGAAAGGACTAGC,0,MM129,16642,0.3425972,0.0006922375,0.1715083,0.7075160,0.2442142,4.512698,⋯,15104,7.274705,22470,7756,48576,3318,22,31759,2,PP1
MM129_AAACGAAAGGCAATTA,0,MM129,18778,0.2855579,0.0001712072,0.2160653,0.6379631,0.2196323,4.677799,⋯,21669,7.231448,29805,10261,65759,4180,8,46719,2,PP1
MM129_AAACGAAAGGCTTCGC,0,MM129,16138,0.3735908,0.0000000000,0.1350667,0.6681628,0.2197411,4.441224,⋯,12965,7.292044,18013,5924,43197,2613,0,26959,2,PP1


[1] TRUE

In [45]:
peak_name=rownames(test.data)
peak_name1=peak_name
for (i in 1:length(peak_name)){
    peak_name1[i]=paste0('chr',peak_name[i])
}
rownames(test.data)=peak_name1
activity.matrix <- CreateGeneActivityMatrix(peak.matrix = test.data, annotation.file = "/projects/sanderlab/gaowei/reference/refdata-cellranger-atac-hg19-1.1.0/genes/genes.gtf", 
   seq.levels = paste0('chr',c(1:22, "X", "Y")), upstream = 2000, verbose = TRUE)

class(test.data)

[1] "dgTMatrix"
attr(,"package")
[1] "Matrix"

In [46]:
pbmc.atac <- CreateSeuratObject(counts = test.data, assay = "ATAC", project = "10x_ATAC")
pbmc.atac[["ACTIVITY"]] <- CreateAssayObject(counts = activity.matrix)


Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”

In [47]:
U=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/SC_snATAC_UMAP.csv')
U_temp=U[as.character(U$index) %in% as.character(B_temp$index),]
all(as.character(U_temp$index)==as.character(B_temp$index))

pbmc.atac[["UMAP1"]] <- as.numeric(U_temp$UMAP_1)
pbmc.atac[["UMAP2"]] <- as.numeric(U_temp$UMAP_2)

[1] TRUE

In [48]:
pbmc.atac[["leiden"]] <- as.character(B_temp$leiden)
pbmc.atac[["donor"]] <- as.character(B_temp$donor)
pbmc.atac[["anno"]] <- as.character(B_temp$anno)


In [49]:
DefaultAssay(pbmc.atac) <- "ACTIVITY"
pbmc.atac <- FindVariableFeatures(pbmc.atac)
pbmc.atac <- NormalizeData(pbmc.atac)

pbmc.atac <- ScaleData(pbmc.atac)

Centering and scaling data matrix


In [50]:
DefaultAssay(pbmc.atac) <- "ATAC"
VariableFeatures(pbmc.atac) <- names(which(Matrix::rowSums(pbmc.atac) > 100))
pbmc.atac <- RunLSI(pbmc.atac, n = 50, scale.max = NULL)
pbmc.atac <- RunUMAP(pbmc.atac, reduction = "lsi", dims = 1:50)

Performing TF-IDF normalization
Running SVD on TF-IDF matrix
Scaling cell embeddings


In [51]:
pbmc.atac@reductions$umap@cell.embeddings[,1] <- pbmc.atac$UMAP1
pbmc.atac@reductions$umap@cell.embeddings[,2] <- pbmc.atac$UMAP2

In [52]:
pbmc.atac@meta.data[1:3,]

,orig.ident,nCount_ATAC,nFeature_ATAC,nCount_ACTIVITY,nFeature_ACTIVITY,UMAP1,UMAP2,leiden,donor,anno
MM129_AAACGAAAGGACTAGC,MM129,9865,9865,7615,5688,2.655208,-13.55851,2,MM129,PP1
MM129_AAACGAAAGGCAATTA,MM129,13087,13087,10046,7016,2.295038,-12.92538,2,MM129,PP1
MM129_AAACGAAAGGCTTCGC,MM129,8093,8093,6096,4766,2.192755,-12.37533,2,MM129,PP1


In [53]:
#saveRDS(pbmc.atac, file = "/oasis/tscc/scratch/gaw006/share/SCislet/SC_ATAC.rds")

## Integration

In [21]:
pbmc.rna <- readRDS("/oasis/tscc/scratch/gaw006/share/SCislet/SC_RNA.rds")
pbmc.rna$tech <- "rna"

pbmc.atac <- readRDS("/oasis/tscc/scratch/gaw006/share/SCislet/SC_ATAC.rds")
pbmc.atac$tech <- "atac"


In [23]:
pbmc.rna@meta.data$anno=paste0('anno_',pbmc.rna@meta.data$anno)
pbmc.rna@meta.data[1:3,]
pbmc.atac@meta.data[1:3,]

,orig.ident,nCount_RNA,nFeature_RNA,leiden,donor,percent.mt,log_count,RNA_snn_res.0.5,seurat_clusters,UMAP1,UMAP2,tech,anno
D11_AAACCCAAGAACTGAT-1,D11,12415,4120,6,D11,7.482884,9.426661,5,5,9.486988,7.8144700,rna,anno_
D11_AAACCCAAGACATCAA-1,D11,11961,4105,0,D11,14.171056,9.389407,1,1,6.949167,-0.5240627,rna,anno_
D11_AAACCCACAAGAGATT-1,D11,8323,3238,0,D11,9.419680,9.026778,1,1,7.134753,-0.5965236,rna,anno_


,orig.ident,nCount_ATAC,nFeature_ATAC,nCount_ACTIVITY,nFeature_ACTIVITY,UMAP1,UMAP2,leiden,donor,anno,tech
MM129_AAACGAAAGGACTAGC,MM129,9865,9865,7615,5688,2.655208,-13.55851,2,MM129,PP1,atac
MM129_AAACGAAAGGCAATTA,MM129,13087,13087,10046,7016,2.295038,-12.92538,2,MM129,PP1,atac
MM129_AAACGAAAGGCTTCGC,MM129,8093,8093,6096,4766,2.192755,-12.37533,2,MM129,PP1,atac


In [24]:
transfer.anchors <- FindTransferAnchors(reference = pbmc.rna, query = pbmc.atac, features = VariableFeatures(object = pbmc.rna), 
    reference.assay = "RNA", query.assay = "ACTIVITY", reduction = "cca")

leiden_ID=as.character(pbmc.rna@meta.data$leiden)
leiden.predictions <- TransferData(anchorset = transfer.anchors, refdata = leiden_ID, weight.reduction = pbmc.atac[["lsi"]])
pbmc.atac <- AddMetaData(pbmc.atac, metadata = leiden.predictions)

pbmc.atac@meta.data$predicted_leiden=pbmc.atac@meta.data$predicted.id


Warning message in RunCCA.Seurat(object1 = reference, object2 = query, features = features, :
“Running CCA on different assays”Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 39006 anchors
Filtering anchors
	Retained 2485 anchors
Extracting within-dataset neighbors
Finding integration vectors
Finding integration vector weights
Predicting cell labels


In [26]:
anno_ID=as.character(pbmc.rna@meta.data$anno)
anno.predictions <- TransferData(anchorset = transfer.anchors, refdata = anno_ID, weight.reduction = pbmc.atac[["lsi"]])
pbmc.atac <- AddMetaData(pbmc.atac, metadata = anno.predictions)

pbmc.atac@meta.data$predicted_anno=pbmc.atac@meta.data$predicted.id

Finding integration vectors
Finding integration vector weights
Predicting cell labels


In [27]:
pbmc.rna@meta.data[1:3,]
pbmc.atac@meta.data[1:3,]

,orig.ident,nCount_RNA,nFeature_RNA,leiden,donor,percent.mt,log_count,RNA_snn_res.0.5,seurat_clusters,UMAP1,UMAP2,tech,anno
D11_AAACCCAAGAACTGAT-1,D11,12415,4120,6,D11,7.482884,9.426661,5,5,9.486988,7.8144700,rna,anno_
D11_AAACCCAAGACATCAA-1,D11,11961,4105,0,D11,14.171056,9.389407,1,1,6.949167,-0.5240627,rna,anno_
D11_AAACCCACAAGAGATT-1,D11,8323,3238,0,D11,9.419680,9.026778,1,1,7.134753,-0.5965236,rna,anno_


,orig.ident,nCount_ATAC,nFeature_ATAC,nCount_ACTIVITY,nFeature_ACTIVITY,UMAP1,UMAP2,leiden,donor,anno,⋯,prediction.score.2,prediction.score.7,prediction.score.5,prediction.score.11,prediction.score.12,prediction.score.10,prediction.score.max,predicted_leiden,prediction.score.anno_,predicted_anno
MM129_AAACGAAAGGACTAGC,MM129,9865,9865,7615,5688,2.655208,-13.55851,2,MM129,PP1,⋯,0.059579196,0.000000000,0.08886732,0.09063191,0,0,1,1,1,anno_
MM129_AAACGAAAGGCAATTA,MM129,13087,13087,10046,7016,2.295038,-12.92538,2,MM129,PP1,⋯,0.003911255,0.000000000,0.03461148,0.00000000,0,0,1,1,1,anno_
MM129_AAACGAAAGGCTTCGC,MM129,8093,8093,6096,4766,2.192755,-12.37533,2,MM129,PP1,⋯,0.001483252,0.001280583,0.01674647,0.00000000,0,0,1,1,1,anno_


In [28]:
refdata <- GetAssayData(pbmc.rna, assay = "RNA", slot = "data")
imputation <- TransferData(anchorset = transfer.anchors, refdata = refdata, weight.reduction = pbmc.atac[["lsi"]],
    dims = 2:30)
pbmc.atac[["RNA"]] <- imputation



Finding integration vectors
Finding integration vector weights
Transfering 32738 features onto reference data


In [29]:
pbmc.atac@reductions$umap@cell.embeddings[,1] <- pbmc.atac$UMAP1
pbmc.atac@reductions$umap@cell.embeddings[,2] <- pbmc.atac$UMAP2
#saveRDS(pbmc.atac, file = "/oasis/tscc/scratch/gaw006/share/SCislet/SC_ATAC_integration.rds")

# Integration of endocrine snATAC-seq and snRNA-seq data

## snRNA-seq data

In [31]:
test.data <- Matrix::readMM('/oasis/tscc/scratch/gaw006/share/SCislet/RNA_all.mtx')
test.gene=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/RNA_all.genes')
test.cell=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/RNA_all.barcodes')
test.data=Matrix::t(test.data)

rownames(test.data)=test.gene$V1
colnames(test.data)=test.cell$V1
dim(test.data)

[1] 32738 72905

In [33]:
B=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/RNA_barcode.csv')
barcode_all=as.character(B$X)
barcode_temp=barcode_all
B_temp=B

test.data1=test.data[,colnames(test.data) %in% barcode_temp]
dim(test.data1)
all(colnames(test.data1)==as.character(B_temp$X))

primary_RNA <- CreateSeuratObject(counts = test.data1, assay = "RNA", project = "10x_RNA")
primary_RNA

primary_RNA[["leiden"]] <- as.character(B_temp$leiden)
primary_RNA[["donor"]] <- as.character(B_temp$donor)
primary_RNA[["anno"]] <- as.character(B_temp$anno)
primary_RNA[["anno1"]] <- as.character(B_temp$anno1)
primary_RNA[["anno2"]] <- as.character(B_temp$anno2)

[1] 32738 72905

[1] TRUE

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”

An object of class Seurat 
32738 features across 72905 samples within 1 assay 
Active assay: RNA (32738 features)

In [34]:
U=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/RNA_UMAP.csv')
U_temp=U[as.character(U$X) %in% as.character(B_temp$X),]
all(as.character(U_temp$X)==as.character(B_temp$X))

primary_RNA[["UMAP1"]] <- as.numeric(U_temp$UMAP_1)
primary_RNA[["UMAP2"]] <- as.numeric(U_temp$UMAP_2)

[1] TRUE

In [35]:
primary_RNA[["percent.mt"]] <- PercentageFeatureSet(primary_RNA, pattern = "^MT-")

primary_RNA <- NormalizeData(primary_RNA, normalization.method = "LogNormalize", scale.factor = 10000)
primary_RNA <- FindVariableFeatures(primary_RNA, selection.method = "vst", nfeatures = 2000)


In [36]:
primary_RNA@meta.data$log_count=log(primary_RNA@meta.data$nCount_RNA)
primary_RNA <- ScaleData(primary_RNA, vars.to.regress = c("percent.mt","log_count"))


Regressing out percent.mt, log_count
Centering and scaling data matrix


In [37]:
primary_RNA <- RunPCA(primary_RNA, features = VariableFeatures(object = primary_RNA))

PC_ 1 
Positive:  FTH1, RPL17, PPDPF, RPL36A, IGFBP7, HLA-B, SCGB2A1, S100A6, HSPB1, GABARAP 
	   VGF, JUNB, SELM, ANXA2, EEF1G, REG1A, PRDX1, NUPR1L, SCG2, C15orf48 
	   RASD1, SIVA1, TMEM141, FXYD5, HLA-E, FXYD2, C6orf1, HOPX, FXYD3, S100A11 
Negative:  DPP10, HS6ST3, PDE4D, SGCD, CACNA2D1, LSAMP, LARGE, ANKS1B, CTNNA2, PRICKLE2 
	   KCNH7, HECW2, MLLT3, ADARB2, FAT3, GALNT13, RP1-34H18.1, RP11-499F3.2, PLCL1, FAM155A 
	   DDC, LINC00907, NEGR1, XKR4, CTC-340A15.2, CDKAL1, UNC5D, AFF3, ADCY2, RAB3C 
PC_ 2 
Positive:  BRINP3, ARPP21, LMX1A, COL5A2, ARHGAP24, KCNK13, TTC29, AFF3, TPH1, ASTN2 
	   MME, GRIA1, SLC18A1, ZMAT4, ADAMTS9, ADAMTSL3, ZBTB7C, SH3RF3, SORCS3, RYR2 
	   SYT6, RERG, PLD5, ASIC2, KCNS3, CRISPLD1, RP11-547I7.1, STAC, ARHGAP20, ADARB2 
Negative:  PDZD2, CNTN4, ASPH, SLC35F4, SOX6, ETV1, MAGI2, ST6GALNAC5, RP11-767I20.1, PSD3 
	   POU6F2, LPAR1, USH2A, LRRTM4, NRXN3, UNC5C, BTBD11, LPPR5, IQGAP2, ESRRG 
	   LINC-PINT, CNTN5, DSCAML1, LINC01146, PITPNC1, SNED1, MAMLD1,

In [38]:
primary_RNA <- FindNeighbors(primary_RNA, dims = 1:20)
primary_RNA <- FindClusters(primary_RNA, resolution = 0.5)
primary_RNA <- RunUMAP(primary_RNA, dims = 1:20)

Computing nearest neighbor graph
Computing SNN


Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 72905
Number of edges: 2506317

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9639
Number of communities: 24
Elapsed time: 24 seconds


In [39]:
primary_RNA@reductions$umap@cell.embeddings[,1] <- primary_RNA$UMAP1
primary_RNA@reductions$umap@cell.embeddings[,2] <- primary_RNA$UMAP2

In [40]:
#saveRDS(primary_RNA, file = "/oasis/tscc/scratch/gaw006/share/SCislet/endocrine_RNA.rds")

## snATAC-seq

### SC-islet

In [35]:
test.data <- Matrix::readMM('/oasis/tscc/scratch/gaw006/share/SCislet/snATACD_all.mtx')
test.data@x[test.data@x > 0] <- 1
test.gene=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/snATACD_all.regions')
test.cell=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/snATACD_all.barcodes')
test.data=Matrix::t(test.data)

rownames(test.data)=test.gene$V1
colnames(test.data)=test.cell$V1
dim(test.data)

[1] 441034  42441

In [36]:
B=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/snATAC_annotation_han.csv')
barcode_all=as.character(B$X)
B[1:3,]

barcode_temp=colnames(test.data)
B_temp=B[barcode_all %in% barcode_temp,]

test.data=test.data[,colnames(test.data) %in% barcode_temp]
all(colnames(test.data)==as.character(B_temp$X))

X,unique_usable_reads,total_sequenced_reads,duplicated_reads,unique_mito_reads,reads_in_peaks,reads_in_promoters,tss_used,frac_reads_in_peaks,frac_reads_in_promoters,⋯,donor,n_peaks,norm_log_counts,log10_usable_counts,log_usable_counts,leiden,test,anno,anno1,anno2
JYH792_AAACGAAAGGGTCTGA,13435,15429,1904,10,6039,2767,1278,0.4494976,0.2059546,⋯,JYH792,6353,7.272557,3.292034,7.580189,1,1,biobank_beta,PA_beta,beta1
JYH792_AAACGAAAGTAGACCG,8771,11271,2062,269,3097,1431,699,0.3530954,0.1631513,⋯,JYH792,4295,7.360786,3.140194,7.230563,0,0,biobank_alpha,PA_alpha,alpha
JYH792_AAACGAACAAATAGTG,5179,6491,1154,89,2393,1099,540,0.4620583,0.2122031,⋯,JYH792,2651,7.486815,2.973128,6.845880,0,0,biobank_alpha,PA_alpha,alpha


[1] TRUE

In [37]:
peak_name=rownames(test.data)
peak_name1=peak_name
for (i in 1:length(peak_name)){
    peak_name1[i]=paste0('chr',peak_name[i])
}
rownames(test.data)=peak_name1
activity.matrix <- CreateGeneActivityMatrix(peak.matrix = test.data, annotation.file = "/projects/sanderlab/gaowei/reference/refdata-cellranger-atac-hg19-1.1.0/genes/genes.gtf", 
   seq.levels = paste0('chr',c(1:22, "X", "Y")), upstream = 2000, verbose = TRUE)

class(test.data)

[1] "dgTMatrix"
attr(,"package")
[1] "Matrix"

In [38]:
pbmc.atac <- CreateSeuratObject(counts = test.data, assay = "ATAC", project = "10x_ATAC")
pbmc.atac[["ACTIVITY"]] <- CreateAssayObject(counts = activity.matrix)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”

In [39]:
U=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/snATAC_endo_UMAP.csv')
U_temp=U[as.character(U$X) %in% as.character(B_temp$X),]

all(as.character(U_temp$X)==as.character(B_temp$X))

pbmc.atac[["UMAP1"]] <- as.numeric(U_temp$UMAP_1)
pbmc.atac[["UMAP2"]] <- as.numeric(U_temp$UMAP_2)

[1] TRUE

In [40]:
pbmc.atac[["leiden"]] <- as.character(B_temp$leiden)
pbmc.atac[["donor"]] <- as.character(B_temp$donor)
pbmc.atac[["anno"]] <- as.character(B_temp$anno)
pbmc.atac[["anno1"]] <- as.character(B_temp$anno1)
pbmc.atac[["anno2"]] <- as.character(B_temp$anno2)

In [41]:
DefaultAssay(pbmc.atac) <- "ACTIVITY"
pbmc.atac <- FindVariableFeatures(pbmc.atac)
pbmc.atac <- NormalizeData(pbmc.atac)

pbmc.atac <- ScaleData(pbmc.atac)

Centering and scaling data matrix


In [42]:
#saveRDS(pbmc.atac, file = "/oasis/tscc/scratch/gaw006/share/SCislet/endocrine_ATAC_SCislet.rds")

## Childhood

In [54]:
test.data <- Matrix::readMM('/oasis/tscc/scratch/gaw006/share/SCislet/snATACJ_all.mtx')
test.data@x[test.data@x > 0] <- 1
test.gene=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/snATACJ_all.regions')
test.cell=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/snATACJ_all.barcodes')
test.data=Matrix::t(test.data)

rownames(test.data)=test.gene$V1
colnames(test.data)=test.cell$V1
dim(test.data)

[1] 441034   6843

In [55]:
B=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/snATAC_annotation_han.csv')
barcode_all=as.character(B$X)
B[1:3,]

barcode_temp=colnames(test.data)
B_temp=B[barcode_all %in% barcode_temp,]

test.data=test.data[,colnames(test.data) %in% barcode_temp]
all(colnames(test.data)==as.character(B_temp$X))

X,unique_usable_reads,total_sequenced_reads,duplicated_reads,unique_mito_reads,reads_in_peaks,reads_in_promoters,tss_used,frac_reads_in_peaks,frac_reads_in_promoters,⋯,donor,n_peaks,norm_log_counts,log10_usable_counts,log_usable_counts,leiden,test,anno,anno1,anno2
JYH792_AAACGAAAGGGTCTGA,13435,15429,1904,10,6039,2767,1278,0.4494976,0.2059546,⋯,JYH792,6353,7.272557,3.292034,7.580189,1,1,biobank_beta,PA_beta,beta1
JYH792_AAACGAAAGTAGACCG,8771,11271,2062,269,3097,1431,699,0.3530954,0.1631513,⋯,JYH792,4295,7.360786,3.140194,7.230563,0,0,biobank_alpha,PA_alpha,alpha
JYH792_AAACGAACAAATAGTG,5179,6491,1154,89,2393,1099,540,0.4620583,0.2122031,⋯,JYH792,2651,7.486815,2.973128,6.845880,0,0,biobank_alpha,PA_alpha,alpha


[1] TRUE

In [56]:
peak_name=rownames(test.data)
peak_name1=peak_name
for (i in 1:length(peak_name)){
    peak_name1[i]=paste0('chr',peak_name[i])
}
rownames(test.data)=peak_name1
activity.matrix <- CreateGeneActivityMatrix(peak.matrix = test.data, annotation.file = "/projects/sanderlab/gaowei/reference/refdata-cellranger-atac-hg19-1.1.0/genes/genes.gtf", 
   seq.levels = paste0('chr',c(1:22, "X", "Y")), upstream = 2000, verbose = TRUE)

class(test.data)

[1] "dgTMatrix"
attr(,"package")
[1] "Matrix"

In [57]:
pbmc.atac <- CreateSeuratObject(counts = test.data, assay = "ATAC", project = "10x_ATAC")
pbmc.atac[["ACTIVITY"]] <- CreateAssayObject(counts = activity.matrix)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”

In [58]:
U=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/snATAC_endo_UMAP.csv')
U_temp=U[as.character(U$X) %in% as.character(B_temp$X),]

all(as.character(U_temp$X)==as.character(B_temp$X))

pbmc.atac[["UMAP1"]] <- as.numeric(U_temp$UMAP_1)
pbmc.atac[["UMAP2"]] <- as.numeric(U_temp$UMAP_2)

[1] TRUE

In [59]:
pbmc.atac[["leiden"]] <- as.character(B_temp$leiden)
pbmc.atac[["donor"]] <- as.character(B_temp$donor)
pbmc.atac[["anno"]] <- as.character(B_temp$anno)
pbmc.atac[["anno1"]] <- as.character(B_temp$anno1)
pbmc.atac[["anno2"]] <- as.character(B_temp$anno2)

In [60]:
DefaultAssay(pbmc.atac) <- "ACTIVITY"
pbmc.atac <- FindVariableFeatures(pbmc.atac)
pbmc.atac <- NormalizeData(pbmc.atac)

pbmc.atac <- ScaleData(pbmc.atac)

Centering and scaling data matrix


In [61]:
#saveRDS(pbmc.atac, file = "/oasis/tscc/scratch/gaw006/share/SCislet/endocrine_ATAC_Childhood.rds")

### Adult

In [25]:
test.data <- Matrix::readMM('/oasis/tscc/scratch/gaw006/share/SCislet/snATACP_all.mtx')
test.data@x[test.data@x > 0] <- 1
test.gene=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/snATACP_all.regions')
test.cell=read.table('/oasis/tscc/scratch/gaw006/share/SCislet/snATACP_all.barcodes')
test.data=Matrix::t(test.data)

rownames(test.data)=test.gene$V1
colnames(test.data)=test.cell$V1
dim(test.data)

test.data=test.data[,1:20000]

[1] 441034  64461

In [26]:
B=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/snATAC_annotation_han.csv')
barcode_all=as.character(B$X)
B[1:3,]

barcode_temp=colnames(test.data)
B_temp=B[barcode_all %in% barcode_temp,]

test.data=test.data[,colnames(test.data) %in% barcode_temp]
all(colnames(test.data)==as.character(B_temp$X))

X,unique_usable_reads,total_sequenced_reads,duplicated_reads,unique_mito_reads,reads_in_peaks,reads_in_promoters,tss_used,frac_reads_in_peaks,frac_reads_in_promoters,⋯,donor,n_peaks,norm_log_counts,log10_usable_counts,log_usable_counts,leiden,test,anno,anno1,anno2
JYH792_AAACGAAAGGGTCTGA,13435,15429,1904,10,6039,2767,1278,0.4494976,0.2059546,⋯,JYH792,6353,7.272557,3.292034,7.580189,1,1,biobank_beta,PA_beta,beta1
JYH792_AAACGAAAGTAGACCG,8771,11271,2062,269,3097,1431,699,0.3530954,0.1631513,⋯,JYH792,4295,7.360786,3.140194,7.230563,0,0,biobank_alpha,PA_alpha,alpha
JYH792_AAACGAACAAATAGTG,5179,6491,1154,89,2393,1099,540,0.4620583,0.2122031,⋯,JYH792,2651,7.486815,2.973128,6.845880,0,0,biobank_alpha,PA_alpha,alpha


[1] TRUE

In [27]:
peak_name=rownames(test.data)
peak_name1=peak_name
for (i in 1:length(peak_name)){
    peak_name1[i]=paste0('chr',peak_name[i])
}
rownames(test.data)=peak_name1
activity.matrix <- CreateGeneActivityMatrix(peak.matrix = test.data, annotation.file = "/projects/sanderlab/gaowei/reference/refdata-cellranger-atac-hg19-1.1.0/genes/genes.gtf", 
   seq.levels = paste0('chr',c(1:22, "X", "Y")), upstream = 2000, verbose = TRUE)

class(test.data)


[1] "dgTMatrix"
attr(,"package")
[1] "Matrix"

In [28]:
pbmc.atac <- CreateSeuratObject(counts = test.data, assay = "ATAC", project = "10x_ATAC")
pbmc.atac[["ACTIVITY"]] <- CreateAssayObject(counts = activity.matrix)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”

In [29]:
U=read.csv('/oasis/tscc/scratch/gaw006/share/SCislet/snATAC_endo_UMAP.csv')
U_temp=U[as.character(U$X) %in% as.character(B_temp$X),]

all(as.character(U_temp$X)==as.character(B_temp$X))

pbmc.atac[["UMAP1"]] <- as.numeric(U_temp$UMAP_1)
pbmc.atac[["UMAP2"]] <- as.numeric(U_temp$UMAP_2)

[1] TRUE

In [30]:
pbmc.atac[["leiden"]] <- as.character(B_temp$leiden)
pbmc.atac[["donor"]] <- as.character(B_temp$donor)
pbmc.atac[["anno"]] <- as.character(B_temp$anno)
pbmc.atac[["anno1"]] <- as.character(B_temp$anno1)
pbmc.atac[["anno2"]] <- as.character(B_temp$anno2)

In [31]:
DefaultAssay(pbmc.atac) <- "ACTIVITY"
pbmc.atac <- FindVariableFeatures(pbmc.atac)
pbmc.atac <- NormalizeData(pbmc.atac)

pbmc.atac <- ScaleData(pbmc.atac)

Centering and scaling data matrix


In [32]:
DefaultAssay(pbmc.atac) <- "ATAC"
VariableFeatures(pbmc.atac) <- names(which(Matrix::rowSums(pbmc.atac) > 100))
pbmc.atac <- RunLSI(pbmc.atac, n = 50, scale.max = NULL)
pbmc.atac <- RunUMAP(pbmc.atac, reduction = "lsi", dims = 1:50)

Performing TF-IDF normalization
Running SVD on TF-IDF matrix
Scaling cell embeddings


In [33]:
pbmc.atac@reductions$umap@cell.embeddings[,1] <- pbmc.atac$UMAP1
pbmc.atac@reductions$umap@cell.embeddings[,2] <- pbmc.atac$UMAP2

In [34]:
#saveRDS(pbmc.atac, file = "/oasis/tscc/scratch/gaw006/share/SCislet/endocrine_ATAC_Adult.rds")

## integration

In [17]:
pbmc.rna <- readRDS("/oasis/tscc/scratch/gaw006/share/SCislet/endocrine_RNA.rds")
pbmc.rna$tech <- "rna"

pbmc.atac <- readRDS("/oasis/tscc/scratch/gaw006/share/SCislet/endocrine_ATAC_Adult.rds")
pbmc.atac$tech <- "atac"

In [18]:
pbmc.rna@meta.data$anno=paste0('anno_',pbmc.rna@meta.data$anno)
pbmc.rna@meta.data$anno1=paste0('anno1_',pbmc.rna@meta.data$anno1)
pbmc.rna@meta.data$anno2=paste0('anno2_',pbmc.rna@meta.data$anno2)
pbmc.rna@meta.data[1:3,]
pbmc.atac@meta.data[1:3,]

,orig.ident,nCount_RNA,nFeature_RNA,leiden,donor,anno,anno1,anno2,UMAP1,UMAP2,percent.mt,log_count,RNA_snn_res.0.5,seurat_clusters,tech
2Y_scRNAseq_rep1_AAAGGGCTCCGTAGGC-1,2Y,16523,3256,0,2Y_scRNAseq_rep1,anno_gamma,anno1_gamma_J,anno2_alpha,0.03960709,0.03791767,4.472553,9.712509,19,19,rna
2Y_scRNAseq_rep1_AAAGTCCTCATTACGG-1,2Y,29804,3695,1,2Y_scRNAseq_rep1,anno_beta,anno1_beta_J,anno2_beta1,10.09771400,0.28391224,6.539391,10.302398,19,19,rna
2Y_scRNAseq_rep1_AAGCATCGTAGTATAG-1,2Y,21927,2823,1,2Y_scRNAseq_rep1,anno_beta,anno1_beta_J,anno2_beta1,9.47727500,2.10515620,4.929995,9.995474,19,19,rna


,orig.ident,nCount_ATAC,nFeature_ATAC,nCount_ACTIVITY,nFeature_ACTIVITY,UMAP1,UMAP2,leiden,donor,anno,anno1,anno2,tech
JYH792_AAACGAAAGGGTCTGA,JYH792,2795,2795,2343,2044,10.9572430,13.211759,1,JYH792,biobank_beta,PA_beta,beta1,atac
JYH792_AAACGAAAGTAGACCG,JYH792,1505,1505,1245,1140,-0.6241122,7.731968,0,JYH792,biobank_alpha,PA_alpha,alpha,atac
JYH792_AAACGAACAAATAGTG,JYH792,1157,1157,972,919,-1.7004145,7.706151,0,JYH792,biobank_alpha,PA_alpha,alpha,atac


In [19]:
transfer.anchors <- FindTransferAnchors(reference = pbmc.rna, query = pbmc.atac, features = VariableFeatures(object = pbmc.rna), 
    reference.assay = "RNA", query.assay = "ACTIVITY", reduction = "cca")

leiden_ID=as.character(pbmc.rna@meta.data$leiden)
leiden.predictions <- TransferData(anchorset = transfer.anchors, refdata = leiden_ID, weight.reduction = pbmc.atac[["lsi"]])
pbmc.atac <- AddMetaData(pbmc.atac, metadata = leiden.predictions)

pbmc.atac@meta.data$predicted_leiden=pbmc.atac@meta.data$predicted.id


Warning message in RunCCA.Seurat(object1 = reference, object2 = query, features = features, :
“Running CCA on different assays”Running CCA
Merging objects
Finding neighborhoods
Finding anchors
	Found 45648 anchors
Filtering anchors
	Retained 3335 anchors
Extracting within-dataset neighbors
Finding integration vectors
Finding integration vector weights
Predicting cell labels


In [20]:
anno_ID=as.character(pbmc.rna@meta.data$anno)
anno.predictions <- TransferData(anchorset = transfer.anchors, refdata = anno_ID, weight.reduction = pbmc.atac[["lsi"]])
pbmc.atac <- AddMetaData(pbmc.atac, metadata = anno.predictions)

pbmc.atac@meta.data$predicted_anno=pbmc.atac@meta.data$predicted.id

Finding integration vectors
Finding integration vector weights
Predicting cell labels


In [21]:
anno1_ID=as.character(pbmc.rna@meta.data$anno1)
anno1.predictions <- TransferData(anchorset = transfer.anchors, refdata = anno1_ID, weight.reduction = pbmc.atac[["lsi"]])
pbmc.atac <- AddMetaData(pbmc.atac, metadata = anno1.predictions)

pbmc.atac@meta.data$predicted_anno1=pbmc.atac@meta.data$predicted.id

Finding integration vectors
Finding integration vector weights
Predicting cell labels


In [22]:
anno2_ID=as.character(pbmc.rna@meta.data$anno2)
anno2.predictions <- TransferData(anchorset = transfer.anchors, refdata = anno2_ID, weight.reduction = pbmc.atac[["lsi"]])
pbmc.atac <- AddMetaData(pbmc.atac, metadata = anno2.predictions)

pbmc.atac@meta.data$predicted_anno2=pbmc.atac@meta.data$predicted.id


Finding integration vectors
Finding integration vector weights
Predicting cell labels


In [23]:
refdata <- GetAssayData(pbmc.rna, assay = "RNA", slot = "data")
imputation <- TransferData(anchorset = transfer.anchors, refdata = refdata, weight.reduction = pbmc.atac[["lsi"]],
    dims = 2:30)
pbmc.atac[["RNA"]] <- imputation

Finding integration vectors
Finding integration vector weights
Transfering 32738 features onto reference data


In [24]:
pbmc.atac@reductions$umap@cell.embeddings[,1] <- pbmc.atac$UMAP1
pbmc.atac@reductions$umap@cell.embeddings[,2] <- pbmc.atac$UMAP2
#saveRDS(pbmc.atac, file = "/oasis/tscc/scratch/gaw006/share/SCislet/endocrine_ATAC_Adult_integration.rds")
